<a href="https://colab.research.google.com/github/hannahweber244/AAML_Code/blob/main/AAML_WolkenSegmentierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SorourMo/38-Cloud-A-Cloud-Segmentation-Dataset

Cloning into '38-Cloud-A-Cloud-Segmentation-Dataset'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 261 (delta 8), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (261/261), 1.70 MiB | 30.09 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
!gdown --id 12fVQCLxyD_sEI8ME8Ltx0YPNz_8FfuJn
!unzip wolken.zip

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: 38-Cloud_training/train_red/red_patch_271_12_by_7_LC08_L1TP_011002_20160620_20170323_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_011247_20160620_20170323_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_029040_20160720_20170222_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_032029_20160420_20170223_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_034034_20160520_20170223_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_045026_20160720_20170221_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_19_LC08_L1TP_047023_20160920_20170221_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_patch_271_13_by_7_LC08_L1TP_061017_20160720_20170223_01_T1.TIF  
  inflating: 38-Cloud_training/train_red/red_pa

In [ ]:
f = open('wolken/README.md')
f.read()

'# 38-Cloud: A Cloud Segmentation Dataset\n*New: An extension to 38-Cloud dataset is released at [here](https://github.com/SorourMo/95-Cloud-An-Extension-to-38-Cloud-Dataset).\n\nThis dataset contains 38 [Landsat 8](https://www.usgs.gov/land-resources/nli/landsat/landsat-8?qt-science_support_page_related_con=0#qt-science_support_page_related_con) scene images and their manually extracted pixel-level ground truths for cloud detection. 38-Cloud dataset is introduced in [[1]](https://arxiv.org/pdf/1901.10077.pdf), yet it is a modification of the dataset in [[2]](https://ieeexplore.ieee.org/document/8547095).  \nThe entire images of these scenes are cropped into multiple 384*384 patches to be proper for deep learning-based semantic segmentation algorithms. There are 8400 patches for training and 9201 patches for testing.\nEach patch has 4 corresponding spectral channels which are Red (band 4), Green (band 3), Blue (band 2), and Near Infrared (band 5). Unlike other computer vision images, t

In [ ]:
import time
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#define unet

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
                
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)        

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        
        self.conv_last = nn.Conv2d(64, n_class, 1)
        
        
    def forward(self, x):
      #to be completed
        conv1 = self.dconv_down1(x)

        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   
        
        x = self.dconv_down4(x)
        
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)
        
        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   
        
        x = self.dconv_up1(x)
        
        out = self.conv_last(x)
        
        return out

      
model = UNet(6).cuda() #6 classes